# Sorting

### Bubble Sort

```fsharp
[| X(i) <= X(i+1) |]
[| 1;3;7;4;2 |]
// ^>^? swap positions for each position
[| 1;3;7;4;2 |] // N
//     ^>^ ? yes
// etc...
[| 1;3;4;2;|7 |] // N - 1
// after 1 iteration, largest value is at the end. 
// Next iteration doesn't need to go all the way to the end
[| 1;3;2;|4;7 |] // N - 2
[| 1;2;|3;4;7 |] // N -3
[| 1;|2;3;4;7 |] // N - (N -1)
// sorted
```

Each iteration results in `N - i` remaining positions to iterate over.
> `(n(n + 1))/2 -> O(N^2 + N) -> O(N^2)`

In [30]:
// Bubble sort (sort in place algorithm)
let bubble_sort (arr : int array) =
    let len = arr.Length - 1
    let rec walk_array (old_arr : int array) (i : int) (stop : int) =
        if i < stop then
            match arr[i] with
            | x when x <= old_arr[i + 1] ->
                walk_array old_arr (i + 1) stop
            | _ ->
                let high = old_arr[i] 
                arr[i] <- old_arr[i + 1]
                arr[i + 1] <- high
                walk_array arr (i + 1) stop
        else ()
    for stop in len .. -1 .. 0 do
        walk_array arr 0 stop

let unsorted_arr = [|6; 2; 4; 3; 7; 1; 3; 88; 12; 1; 1; 2; |]
bubble_sort unsorted_arr
unsorted_arr // now sorted!

[ 1, 1, 1, 2, 2, 3, 3, 4, 6, 7, 12, 88 ]

In [29]:
// Primeagen's implementation of bubble sort
let bubble_sort (arr : int array) =
    let len = arr.Length - 1
    for i in 0 .. len do
        // never thought to use `i` in the next nested loop
        for j in 0 .. (len - 1 - i) do // -1 since indexing up 1 with arr[j + 1]
            if (arr[j] > arr[j + 1]) then
                let high = arr[j]
                arr[j] <- arr[j + 1]
                arr[j + 1] <- high

let unsorted_arr = [|6; 2; 4; 3; 7; 1; 3; 88; 12; 1; 1; 2; |]
bubble_sort unsorted_arr
unsorted_arr // now sorted!

[ 1, 1, 1, 2, 2, 3, 3, 4, 6, 7, 12, 88 ]